In [1]:
from __future__ import division
from tensorflow import keras
print( 'Using Keras version', keras.__version__)
from tensorflow.keras.datasets import mnist
import os
import tensorflow as tf
import cv2
import random
import gc
import numpy as np
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from time import time

Using Keras version 2.1.6-tf


In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_valid = np.load('x_valid.npy')
y_valid = np.load('y_valid.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

#Normalize data
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_valid = x_valid / 255
x_test = x_test / 255

from tensorflow.keras.utils import to_categorical 
y_train = to_categorical(y_train, 10)
y_valid = to_categorical(y_valid, 10)
y_test = to_categorical(y_test, 10)

#MNIST resolution
img_rows, img_cols, channels = 32, 32, 3

from tensorflow.keras import backend as K
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], channels, img_rows, img_cols)
    x_valid = x_valid.reshape(x_valid.shape[0], channels, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
    x_valid = x_valid.reshape(x_valid.shape[0], img_rows, img_cols, channels)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)

In [3]:
classes = sorted(os.walk('./train/').__next__()[1])
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


# Feed forward NN

In [3]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
#Two hidden layers
nn = Sequential()
nn.add(Dense(64,activation='relu',input_shape=(3072,)))
nn.add(Dense(32,activation='relu'))
nn.add(Dense(32,activation='relu'))
nn.add(Dense(10, activation='softmax'))

#Model visualization
#We can plot the model by using the ```plot_model``` function. We need to install *pydot, graphviz and pydot-ng*.
#from keras.util import plot_model
#plot_model(nn, to_file='nn.png', show_shapes=true)

#Compile the NN
nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

nn.summary()

Number of train examples 90000
Size of train examples (32, 32, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                196672    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 200,138
Trainable params: 200,138
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=20, verbose=1, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.savefig('mnist_fnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.savefig('mnist_fnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

# #Saving model and weights
# from keras.models import model_from_json
# nn_json = nn.to_json()
# with open('nn.json', 'w') as json_file:
#         json_file.write(nn_json)
# weights_file = "weights-MNIST_"+str(score[1])+".hdf5"
# nn.save_weights(weights_file,overwrite=True)

# #Loading model and weights
# json_file = open('nn.json','r')
# nn_json = json_file.read()
# json_file.close()
# nn = model_from_json(nn_json)
# nn.load_weights(weights_file)


# Convolutional NN

In [5]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
#Two hidden layers
nn = Sequential()
nn.add(Conv2D(64, (3,3), activation='relu', input_shape=input_shape))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Conv2D(32, (3,3), activation='relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Flatten())
nn.add(Dense(16, activation='relu'))
nn.add(Dense(10, activation='softmax'))

nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                18448     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                170       
Total para

In [4]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=1000, validation_data=(x_valid, y_valid), verbose=0, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])


test loss: 1.399267485385471
test accuracy: 0.5140555555555556


In [8]:
##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('1_cnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('1_cnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

Analysis of results
              precision    recall  f1-score   support

    airplane       0.81      0.50      0.62      9000
  automobile       0.77      0.36      0.49      9000
        bird       0.54      0.47      0.50      9000
         cat       0.44      0.36      0.40      9000
        deer       0.33      0.71      0.45      9000
         dog       0.40      0.44      0.42      9000
        frog       0.87      0.47      0.61      9000
       horse       0.61      0.63      0.62      9000
        ship       0.84      0.36      0.51      9000
       truck       0.43      0.83      0.57      9000

   micro avg       0.51      0.51      0.51     90000
   macro avg       0.60      0.51      0.52     90000
weighted avg       0.60      0.51      0.52     90000

[[4501  151  586  114  968  349   19  418  367 1527]
 [  63 3260  109  112  317  198   22  233   75 4611]
 [ 230   31 4243  583 2251  880  198  328   67  189]
 [  15   19  528 3284 2215 2125  198  361   15  240]
 [  41   

# Convolutional 2

In [3]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
#Two hidden layers
nn = Sequential()
nn.add(Conv2D(64, (5,5), activation='relu', input_shape=input_shape))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Conv2D(32, (3,3), activation='relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Flatten())
nn.add(Dense(16, activation='relu'))
nn.add(Dense(10, activation='softmax'))

nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                18448     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                170       
Total para

In [5]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=1000, validation_data=(x_valid, y_valid), verbose=0, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])


test loss: 1.2614434773551093
test accuracy: 0.5626111111111111


In [6]:
##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('2_cnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('2_cnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

Analysis of results
              precision    recall  f1-score   support

           0       0.66      0.72      0.69      9000
           1       0.69      0.65      0.67      9000
           2       0.61      0.39      0.48      9000
           3       0.47      0.30      0.37      9000
           4       0.37      0.65      0.47      9000
           5       0.59      0.21      0.31      9000
           6       0.53      0.83      0.65      9000
           7       0.62      0.62      0.62      9000
           8       0.57      0.73      0.64      9000
           9       0.75      0.51      0.61      9000

   micro avg       0.56      0.56      0.56     90000
   macro avg       0.59      0.56      0.55     90000
weighted avg       0.59      0.56      0.55     90000

[[6505  139  250   44  352   24  115  149 1327   95]
 [ 525 5810   78   96  219   35  162  214  914  947]
 [ 689   57 3535  365 1752  221 1475  294  590   22]
 [ 126   84  405 2701 1961  561 2338  396  362   66]
 [ 215   

# Convolutional 3

In [4]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

nn = Sequential()
nn.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
nn.add(Activation('relu'))
nn.add(Conv2D(32, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.25))

nn.add(Conv2D(64, (3, 3), padding='same'))
nn.add(Activation('relu'))
nn.add(Conv2D(64, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.25))

nn.add(Flatten())
nn.add(Dense(512))
nn.add(Activation('relu'))
nn.add(Dropout(0.5))
nn.add(Dense(10))
nn.add(Activation('softmax'))

nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [4]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=1000, validation_data=(x_valid, y_valid), verbose=0, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

test loss: 0.9602733841896057
test accuracy: 0.6569


In [5]:
##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('3_cnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('3_cnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

Analysis of results
              precision    recall  f1-score   support

           0       0.76      0.76      0.76      9000
           1       0.76      0.69      0.72      9000
           2       0.64      0.57      0.60      9000
           3       0.48      0.53      0.50      9000
           4       0.63      0.49      0.55      9000
           5       0.51      0.47      0.49      9000
           6       0.74      0.78      0.76      9000
           7       0.73      0.72      0.72      9000
           8       0.68      0.78      0.73      9000
           9       0.65      0.78      0.71      9000

   micro avg       0.66      0.66      0.66     90000
   macro avg       0.66      0.66      0.65     90000
weighted avg       0.66      0.66      0.65     90000

[[6810  198  228   96   61   61   47  111  998  390]
 [ 229 6184   46   57   17   59   30   66  332 1980]
 [ 587   51 5153  735  457  508  794  167  463   85]
 [ 100   79  507 4791  533 1571  813  199  231  176]
 [ 188   

# Convolutional 4
without regularizations

In [3]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

nn = Sequential()
nn.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
nn.add(Activation('relu'))
nn.add(Conv2D(32, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))

nn.add(Conv2D(64, (3, 3), padding='same'))
nn.add(Activation('relu'))
nn.add(Conv2D(64, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))

nn.add(Flatten())
nn.add(Dense(512))
nn.add(Activation('relu'))
nn.add(Dense(10))
nn.add(Activation('softmax'))

nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

In [4]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=1000, validation_data=(x_valid, y_valid), verbose=0, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

KeyboardInterrupt: 

In [ ]:
##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('4_cnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('4_cnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

# Convolutional 5
with different optimizers -ADAM


In [3]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

nn = Sequential()
nn.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
nn.add(Activation('relu'))
nn.add(Conv2D(32, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.25))

nn.add(Conv2D(64, (3, 3), padding='same'))
nn.add(Activation('relu'))
nn.add(Conv2D(64, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.25))

nn.add(Flatten())
nn.add(Dense(512))
nn.add(Activation('relu'))
nn.add(Dropout(0.5))
nn.add(Dense(10))
nn.add(Activation('softmax'))

from tensorflow.keras.optimizers import Adam

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [4]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=1000, validation_data=(x_valid, y_valid), verbose=0, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

test loss: 0.9499173873583475
test accuracy: 0.6715666666666666


In [5]:
##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('5_cnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('5_cnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

Analysis of results
              precision    recall  f1-score   support

           0       0.81      0.75      0.78      9000
           1       0.76      0.74      0.75      9000
           2       0.70      0.54      0.61      9000
           3       0.54      0.48      0.51      9000
           4       0.52      0.69      0.59      9000
           5       0.53      0.51      0.52      9000
           6       0.67      0.84      0.75      9000
           7       0.78      0.69      0.73      9000
           8       0.71      0.78      0.74      9000
           9       0.76      0.69      0.73      9000

   micro avg       0.67      0.67      0.67     90000
   macro avg       0.68      0.67      0.67     90000
weighted avg       0.68      0.67      0.67     90000

[[6770  139  363  101  156   88   97  166  940  180]
 [ 173 6700   55   45   74  102   83  100  476 1192]
 [ 356   37 4883  596 1035  563 1098  111  291   30]
 [  65   45  315 4342 1144 1465 1267  143  152   62]
 [ 105   

# Convolutional 6
with RMSprop

In [3]:
#Define the NN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

nn = Sequential()
nn.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
nn.add(Activation('relu'))
nn.add(Conv2D(32, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.25))

nn.add(Conv2D(64, (3, 3), padding='same'))
nn.add(Activation('relu'))
nn.add(Conv2D(64, (3, 3)))
nn.add(Activation('relu'))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.25))

nn.add(Flatten())
nn.add(Dense(512))
nn.add(Activation('relu'))
nn.add(Dropout(0.5))
nn.add(Dense(10))
nn.add(Activation('softmax'))

from tensorflow.keras.optimizers import RMSprop

opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

nn.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

#Starting tensorboard
cbacks = []
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
cbacks.append(tensorboard)

modfile = './model%d.h5' % int(time())
mcheck = ModelCheckpoint(filepath=modfile, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
cbacks.append(mcheck)

early = EarlyStopping(monitor='val_loss', patience=10)

cbacks.append(early)

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [4]:
#Start training
history = nn.fit(x_train,y_train,batch_size=128,epochs=1000, validation_data=(x_valid, y_valid), verbose=1, callbacks=cbacks)

#Evaluate the model with test set
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

Train on 90000 samples, validate on 90000 samples
Epoch 1/1000
12416/90000 [===>..........................] - ETA: 1:53 - loss: 2.1527 - acc: 0.2089WARNING:tensorflow:Method on_batch_end() is slow compared to the batch update (0.139293). Check your callbacks.


74368/90000 [=======================>......] - ETA: 1:13 - loss: 1.8145 - acc: 0.3326WARNING:tensorflow:Method on_batch_end() is slow compared to the batch update (0.129055). Check your callbacks.


88832/90000 [============================>.] - ETA: 4s - loss: 1.7715 - acc: 0.3481WARNING:tensorflow:Method on_batch_end() is slow compared to the batch update (0.100257). Check your callbacks.


90000/90000 [==============================] - 369s 4ms/step - loss: 1.7690 - acc: 0.3491 - val_loss: 1.8091 - val_acc: 0.3667
Epoch 2/1000
90000/90000 [==============================] - 45s 495us/step - loss: 1.4302 - acc: 0.4785 - val_loss: 1.7106 - val_acc: 0.4060s - loss:  - ETA: 0s - loss: 1.4304 - acc: 0.
Epoch 3/1000
90000/90000 [==============================] - 39s 435us/step - loss: 1.2945 - acc: 0.5304 - val_loss: 1.2789 - val_acc: 0.5361
Epoch 4/1000
90000/90000 [==============================] - 39s 432us/step - loss: 1.2164 - acc: 0.5626 - val_loss: 1.2179 - val_acc: 0.5695
Epoch 5/1000
90000/90000 [==============================] - 38s 426us/step - loss: 1.1639 - acc: 0.5842 - val_loss: 1.2392 - val_acc: 0.54893 - acc:  - ETA: 24s - lo - ETA: 20s - loss: 1.1656 - acc: 0.58 - ETA: 20s - loss:  - E - ETA: 3s - loss: 1.1662 - acc: 0 - ETA: 2s - loss: 1.1659 - acc: 0. - ETA: 2s - loss: 1.1656 - acc:  - ETA: 1s - loss: 1.1656 - acc: 0.5 - ETA: 1s - loss: 1.1657 - acc: 0.5 - E

In [5]:
##Store Plots
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('6_cnn_accuracy.pdf')
plt.close()
#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('6_cnn_loss.pdf')

#Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
#Compute probabilities
Y_pred = nn.predict(x_test)
#Assign most probable label
y_pred = np.argmax(Y_pred, axis=1)
#Plot statistics
print( 'Analysis of results' )
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

Analysis of results
              precision    recall  f1-score   support

           0       0.58      0.86      0.69      9000
           1       0.70      0.76      0.73      9000
           2       0.57      0.55      0.56      9000
           3       0.47      0.46      0.47      9000
           4       0.52      0.51      0.51      9000
           5       0.50      0.39      0.44      9000
           6       0.92      0.43      0.59      9000
           7       0.53      0.82      0.64      9000
           8       0.74      0.65      0.69      9000
           9       0.77      0.61      0.68      9000

   micro avg       0.61      0.61      0.61     90000
   macro avg       0.63      0.61      0.60     90000
weighted avg       0.63      0.61      0.60     90000

[[7734  162  180   52   46   37    5  196  502   86]
 [ 513 6879   48   38   15   52    0  207  279  969]
 [1072   68 4905  633  860  473  103  636  229   21]
 [ 338   89  558 4157  963 1611  124  945  148   67]
 [ 476   